In [1]:
import praw
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('.env')

True

In [2]:
# Access and use the environment variables
SECRET_KEY = os.getenv("SECRET_KEY")
USER_AGENT = os.getenv("USER_AGENT")
CLIENT_ID = os.getenv("CLIENT_ID")

In [3]:
#Initializing reddit instance

reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=SECRET_KEY,
    user_agent=USER_AGENT,
)

print(reddit.read_only)

True


In [4]:
redditPost = reddit.subreddit("askreddit")
top_posts = redditPost.top(limit=5, time_filter="day")

In [5]:
def getCommentForPost(comments):
    commentList = []

    for comment in comments:
        if ( len(commentList) > 5 ):
            break

        if ( len(comment.body) < 10 or len(comment.body) > 200 ):
            continue
        
        comment_dic = {
            'id': comment.id,
            'text': comment.body
        }

        commentList.append(comment_dic)

    return commentList


In [6]:
# Convert individual posts to dictionaries
post_list = []
for post in top_posts:
    if post.over_18:
        continue

    post_dict = {
        'title': post.title,
        'score': post.score,
        'url': post.url,
        'id': post.id,
        'comments': getCommentForPost(post.comments)
        # Include other desired fields here
    }
    post_list.append(post_dict)

# Convert the list of posts to JSON format

In [7]:
print(post_list)

[{'title': 'Adults of reddit, what is something every teenager should know about "the real world"?', 'score': 15446, 'url': 'https://www.reddit.com/r/AskReddit/comments/14q1o86/adults_of_reddit_what_is_something_every_teenager/', 'id': '14q1o86', 'comments': [{'id': 'jqlv9c6', 'text': "PEOPLE LIE. Just because they're an adult doesn't mean they always tell the truth."}, {'id': 'jqlzfjn', 'text': '$1000 is a lot to owe but not a lot to have…'}, {'id': 'jqlff89', 'text': 'Life never “settles down”. If something is important to you then you should get to it ASAP.'}, {'id': 'jqlrako', 'text': 'For better or for worse, confidence opens as many doors as competence.'}, {'id': 'jqkztxm', 'text': 'Everything is expensive when you move out'}, {'id': 'jqlwf10', 'text': 'You know how sometimes you think to yourself "when I\'m an adult and have my own place I\'ll be able to do whatever I want"? \n\nMake sure whatever you want includes doing the dishes'}]}, {'title': 'Men of Reddit, if you burped in

In [8]:
for post in post_list:
    print(post)

{'title': 'Adults of reddit, what is something every teenager should know about "the real world"?', 'score': 15446, 'url': 'https://www.reddit.com/r/AskReddit/comments/14q1o86/adults_of_reddit_what_is_something_every_teenager/', 'id': '14q1o86', 'comments': [{'id': 'jqlv9c6', 'text': "PEOPLE LIE. Just because they're an adult doesn't mean they always tell the truth."}, {'id': 'jqlzfjn', 'text': '$1000 is a lot to owe but not a lot to have…'}, {'id': 'jqlff89', 'text': 'Life never “settles down”. If something is important to you then you should get to it ASAP.'}, {'id': 'jqlrako', 'text': 'For better or for worse, confidence opens as many doors as competence.'}, {'id': 'jqkztxm', 'text': 'Everything is expensive when you move out'}, {'id': 'jqlwf10', 'text': 'You know how sometimes you think to yourself "when I\'m an adult and have my own place I\'ll be able to do whatever I want"? \n\nMake sure whatever you want includes doing the dishes'}]}
{'title': 'Men of Reddit, if you burped in f

In [9]:
for i, post in enumerate(post_list):
    print(post['title'])

Adults of reddit, what is something every teenager should know about "the real world"?
Men of Reddit, if you burped in front of your girlfriend and she responded back with a louder burp, how would you react?
What's the worst food to get hit by in a food fight?
What is the worst thing your EX said during the breakup?


In [10]:
from controllers.voice_over import createVoiceOver, createVoiceOverPost

for i, post in enumerate(post_list):
    createVoiceOverPost(i, post['title'])
    
    for comment in post['comments']:
        createVoiceOver(comment['id'], comment['text'], i)


In [11]:
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from constants import SS_HEIGHT, SS_WIDTH, screenshotDir
import os

screenWidth = SS_WIDTH
screenHeight = SS_HEIGHT

def takingSS(driver, wait, target_id, fileName):
    print('target_id', target_id)
    element = wait.until(EC.presence_of_element_located((By.ID, target_id)))
    
    try:
        closeId = 'close'
        ele = driver.find_element(By.ID, closeId)
        ele.click()
        print("Element found and clicked!")
    except NoSuchElementException:
        print("Element not found in the DOM.")

    driver.execute_script("window.focus();")
    ss = f'{fileName}.png'
    fp = open(ss, "wb")
    fp.write(element.screenshot_as_png)
    fp.close()

def initiateDriver(post):
    options = webdriver.ChromeOptions()
    options.enable_mobile = False
    options.add_argument('-headless')
    driver = webdriver.Chrome(options=options)
    url = post['url']
    driver.get(url)
    
    wait = WebDriverWait(driver, 10)
    driver.set_window_size(width=screenWidth, height=screenHeight)

    return driver, wait

for i, post in enumerate(post_list):
    driver, wait = initiateDriver(post)
    fileName = f'screenshots/post-{i}'
    os.mkdir(fileName)
    prefix = f'{fileName}/main'
    post_id = f"t3_{post['id']}"
    takingSS(driver, wait, post_id, prefix)
    driver.quit()

target_id t3_14q1o86
Element not found in the DOM.
target_id t3_14pza4f
Element not found in the DOM.
target_id t3_14qdyye
Element not found in the DOM.
target_id t3_14q3bh2
Element not found in the DOM.


In [12]:
from PIL import Image, ImageDraw, ImageFont
import textwrap

def makeScreenShotsForComments(text, path, font_size = 14):
    font_color = (0, 0, 0)  # RGB color for the font
    font_path = "font\Roboto-Black.ttf"  # Replace with the path to your desired font file

    # Create a new image with a white background
    background_color = (255, 255, 255)  # RGB color for the background
    width = 400
    padding = 10  # Padding around the text
    line_spacing = 4  # Spacing between lines of text

    font = ImageFont.truetype(font_path, font_size)
    text_lines = textwrap.wrap(text, width=60)  # Adjust the width parameter as desired

    # Calculate the height of the image based on the text length
    line_height = font.getsize("Sample")[1]
    text_height = (line_height + line_spacing) * len(text_lines)
    height = text_height + (padding * 2)

    image = Image.new("RGB", (width, height), background_color)
    draw = ImageDraw.Draw(image)

    # Calculate the starting position for the text
    text_x = padding
    text_y = padding

    # Draw the text onto the image
    for line in text_lines:
        line_width, line_height = font.getsize(line)
        draw.text((text_x, text_y), line, font=font, fill=font_color)
        text_y += line_height + line_spacing

    image.save(f"{path}.png")
    

In [13]:
for i, post in enumerate(post_list):
    
    for comment in post['comments']:
        path = f"screenshots/post-{i}/comment-{comment['id']}"
        makeScreenShotsForComments(comment['text'], path)


C:\Users\acer\AppData\Local\Temp\ipykernel_3684\3021440192.py:18: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  line_height = font.getsize("Sample")[1]
C:\Users\acer\AppData\Local\Temp\ipykernel_3684\3021440192.py:31: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  line_width, line_height = font.getsize(line)


In [23]:
from moviepy.editor import ImageClip, AudioFileClip, VideoFileClip, CompositeVideoClip
import random

def createVideoClip(ssFile, voiceFile):
    audioClip = AudioFileClip(voiceFile)
    imageClip = ImageClip(ssFile).set_duration(audioClip.duration)
    videoClip = imageClip.set_audio(audioClip)
    return videoClip

for i, post in enumerate(post_list):
    clips = []
    mainPostSS = f"screenshots/post-{i}/main.png"
    mainPostAudio = f"VoiceOvers/post-{i}/main.mp3"
    mainPostClip = createVideoClip(mainPostSS, mainPostAudio)

    clips.append(mainPostClip)

    start_time = mainPostClip.duration  # Set the start time for the next clip

    for comment in post['comments']:
        pathSS = f"screenshots/post-{i}/comment-{comment['id']}.png"
        pathAudio = f"VoiceOvers/post-{i}/comment-{comment['id']}.mp3"
        commentClip = createVideoClip(pathSS, pathAudio)
        commentClip = commentClip.set_start(start_time)  # Set the start time for the comment clip
        clips.append(commentClip)

        start_time += commentClip.duration  # Update the start time for the next clip

    bgIndex = random.randint(1, 4)
    bgVideoPath = f"sampleBgVideo/{bgIndex}.mp4"
    bgClip = VideoFileClip(bgVideoPath)

    finalVideo = CompositeVideoClip(clips=[bgClip] + clips)

    # outputFile = f"finalVideo/reel-{i}.mp4"
    outputFile = 'test-output.mp4'
    finalVideo.write_videofile(outputFile, codec='mpeg4', threads=12, bitrate='8000k')

    break


Moviepy - Building video test-output.mp4.
MoviePy - Writing audio in test-outputTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video test-output.mp4



Moviepy - Done !
Moviepy - video ready test-output.mp4


In [22]:
from moviepy.editor import ImageClip, AudioFileClip, VideoFileClip, concatenate_videoclips
import random

def createVideoClip(ssFile, voiceFile):
    audioClip = AudioFileClip(voiceFile)
    imageClip = ImageClip(ssFile).set_duration(audioClip.duration)
    videoClip = imageClip.set_audio(audioClip)
    return videoClip

for i, post in enumerate(post_list):
    clips = []
    mainPostSS = f"screenshots/post-{i}/main.png"
    mainPostAudio = f"VoiceOvers/post-{i}/main.mp3"
    mainPostClip = createVideoClip(mainPostSS, mainPostAudio)
    clips.append(mainPostClip)

    start_time = mainPostClip.duration

    for comment in post['comments']:
        pathSS = f"screenshots/post-{i}/comment-{comment['id']}.png"
        pathAudio = f"VoiceOvers/post-{i}/comment-{comment['id']}.mp3"
        commentClip = createVideoClip(pathSS, pathAudio)
        commentClip = commentClip.set_start(start_time)
        clips.append(commentClip)
        start_time += commentClip.duration

    bgIndex = random.randint(1, 4)
    bgVideoPath = f"sampleBgVideo/{bgIndex}.mp4"
    bgClip = VideoFileClip(bgVideoPath)

    # Set the duration of the background clip
    total_duration = sum(clip.duration for clip in clips)
    bgClip = bgClip.set_duration(total_duration)

    finalVideo = concatenate_videoclips(clips=[bgClip] + clips, method="compose")

    outputFile = f"finalVideo/reel-{i}.mp4"
    finalVideo.write_videofile(outputFile, codec='mpeg4', threads=12, bitrate='8000k')

    break


Moviepy - Building video finalVideo/reel-0.mp4.
MoviePy - Writing audio in reel-0TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video finalVideo/reel-0.mp4



Moviepy - Done !
Moviepy - video ready finalVideo/reel-0.mp4
